In [4]:
import os
import numpy as np
import pandas as pd
from PIL import Image

In [26]:
df_labels = pd.read_csv("labels.csv")

# define function for collect agent/patient position
def extract_positions_from_maps(maps_path):
    maps = np.array(Image.open(maps_path))

    if maps.shape[-1] == 4:
        maps = maps[..., :3]
        
    agent_maps = np.all(maps == [255, 0, 0], axis=-1) # RED
    patient_maps = np.all(maps == [255, 255, 0], axis=-1) # YELLOW

    agent_pos = np.argwhere(agent_maps)
    patient_pos = np.argwhere(patient_maps)

    agent_center = agent_pos.mean(axis=0) if len(agent_pos) > 0 else None
    patient_center = patient_pos.mean(axis=0) if len(patient_pos) > 0 else None

    return agent_center, patient_center

In [41]:
# traverse all PNG segmentation maps for each test stimulus, collect agent/patient position and ground truth outcome from labels.csv
records = []
base_dir = "."
scenarios = [f for f in os.listdir(base_dir) if os.path.isdir(f) and f[0].isupper()]

for scenario in scenarios:
    maps_dir = os.path.join(base_dir, scenario, "maps")
    if not os.path.isdir(maps_dir):
        continue
    for fname in os.listdir(maps_dir):
        if not fname.endswith(".png"):
            continue
        stim_id = fname.replace("map.png", "img")
        full_path = os.path.join(maps_dir, fname)
        # collect agent/patient position
        agent, patient = extract_positions_from_maps(full_path)
        if agent is None or patient is None:
            continue
        dist = np.linalg.norm(agent - patient) # Compute the Euclidean distance between the center of the agent and the center of the patient

        # distance threshold
        is_interaction = dist < 100  # i.e. 50 pixels, Adjustable.
        
        # matching two dataframes
        match = df_labels[df_labels.iloc[:, 0].astype(str).str.contains(stim_id)]
        if not match.empty:
            row = match.iloc[0]
            records.append({
                "stimulus_id": stim_id,
                "scenario": scenario,
                "agent(RED)_x": agent[1],
                "agent(RED)_y": agent[0],
                "patient(YELLOW)_x": patient[1],
                "patient(YELLOW)_y": patient[0],
                "distance": dist,
                "is_interaction(distance<50 pixels)": is_interaction, 
                "ground truth outcome": row.get("ground truth outcome", None),
            })

df_all = pd.DataFrame(records)

In [42]:
df_all.head()

,stimulus_id,scenario,agent(RED)_x,agent(RED)_y,patient(YELLOW)_x,patient(YELLOW)_y,distance,is_interaction(distance<50 pixels),ground truth outcome
0,pilot-containment-vase_0012_img,Contain,258.634048,199.049598,270.175078,356.040684,157.414728,False,False
1,pilot-containment-vase_0002_img,Contain,254.857357,211.857357,255.490358,383.800866,171.944674,False,True
2,pilot-containment-bowl_0028_img,Contain,250.915220,249.158879,246.260977,352.918477,103.863932,False,True
3,pilot-containment-multi-bowl_0007_img,Contain,263.411282,232.296923,252.872093,322.564718,90.880962,True,False
4,pilot-containment-cylinder-bowl_0024_img,Contain,260.500559,191.652886,253.257683,357.766955,166.271895,False,True
